In [102]:
import pandas as pd
import numpy as np
from loaders import *
import math
import random

In [94]:
matrix_size = 4096

dataflow_permuations = [['M', 'N', 'K'], ['M', 'K', 'N'], ['N', 'K', 'M'], ['N', 'M', 'K'], ['K', 'N', 'M'], ['K', 'M', 'N']]
'''
6 different dataflow permutations
9 components for dataflow
= 6^9 = 10077696 possible combinations
'''
# generate all the data flow permuations
def get_dataflow_permutations():
    dataflow_perumations = []
    
    for DRAM in dataflow_permuations:
        for ClusterArray in dataflow_permuations:
            for GLB_Cluster in dataflow_permuations:
                for glb in dataflow_permuations:
                    for PE_Cluster in dataflow_permuations:
                        for iact_spad in dataflow_permuations:
                            for weight_spad in dataflow_permuations:
                                for psum in dataflow_permuations:
                                    for reg in dataflow_permuations:
                                        permutation = {
                                            "DRAM": DRAM,
                                            "ClusterArray": ClusterArray,
                                            # "GLB_Cluster": GLB_Cluster,
                                            # "glb": glb,
                                            "PE_Cluster": PE_Cluster,
                                            "iact_spad": iact_spad,
                                            "weight_spad": weight_spad,
                                            "psum": psum,
                                            "reg": reg,
                                        }
                                        dataflow_perumations.append(permutation)
    return dataflow_perumations

# generate all the possible factor permutations for a specific rank (ie, M, N, or K)
def get_factor_permutations():
    factor_permutations = []
    
    for DRAM in [2**i for i in range(int(math.log2(matrix_size)) + 1)]:
        curr_prod1 = DRAM
        # for ClusterArray in [2**i for i in range(int(math.log2(matrix_size // curr_prod1)) + 1)]:
        #     curr_prod2 = curr_prod1 * ClusterArray
        #     for GLB_Cluster in [2**i for i in range(int(math.log2(matrix_size // curr_prod2)) + 1)]:
        #         curr_prod3 = curr_prod2 * GLB_Cluster
        #     for glb in [2**i for i in range(int(math.log2(matrix_size // curr_prod2)) + 1)]:
        #         glb = 1
        #     curr_prod3 = curr_prod2 * glb
        #     for PE_Cluster in [2**i for i in range(int(math.log2(matrix_size // curr_prod3)) + 1)]:
        #         curr_prod4 = curr_prod3 * PE_Cluster
        for iact_spad in [2**i for i in range(int(math.log2(matrix_size // curr_prod1)) + 1)]:
            curr_prod2 = curr_prod1 * iact_spad
            for weight_spad in [2**i for i in range(int(math.log2(matrix_size // curr_prod2)) + 1)]:
                curr_prod3 = curr_prod2 * weight_spad
                for psum in [2**i for i in range(int(math.log2(matrix_size // curr_prod3)) + 1)]:
                    final_prod = curr_prod3 * psum
                    if final_prod == matrix_size:
                        permuation = {
                            "DRAM": DRAM,
                            "ClusterArray": 1,
                            # "GLB_Cluster": 1,
                            # "glb": 1,
                            "PE_Cluster": 1,
                            "iact_spad": iact_spad,
                            "weight_spad": weight_spad,
                            "psum": psum,
                            "reg": 1,
                        }
                        factor_permutations.append(permuation)
                                            
    return factor_permutations

In [95]:
dataflow_permutations = get_dataflow_permutations()
factor_permutations = get_factor_permutations()

In [96]:
print("factor_permutations: ", factor_permutations)

factor_permutations:  [{'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 1, 'psum': 4096, 'reg': 1}, {'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 2, 'psum': 2048, 'reg': 1}, {'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 4, 'psum': 1024, 'reg': 1}, {'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 8, 'psum': 512, 'reg': 1}, {'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 16, 'psum': 256, 'reg': 1}, {'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 32, 'psum': 128, 'reg': 1}, {'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 64, 'psum': 64, 'reg': 1}, {'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 128, 'psum': 32, 'reg': 1}, {'DRAM': 1, 'ClusterArray': 1, 'PE_Cluster': 1, 'iact_spad': 1, 'weight_spad': 256, 'psum': 16, 'reg': 1}, {'DRAM': 1, 'Clu

In [97]:
def evaluate_config(config):
    print(config)
    out = run_timeloop_model(
        config,
        problem='designs/baseline/baseline_problem.yaml',
        mapping='designs/baseline/baseline_mapping.yaml',
        sparse_optimizations='designs/baseline/baseline_sparse_opt.yaml', 
    )
    stats = open('./output_dir/timeloop-model.stats.txt', 'r').read()
    print(stats)

def hash_config(config):
    return tuple((k, tuple(v) if isinstance(v, list) else v) for k, v in config.items())

def sample_config(dataflow_permutations, factor_permutations):
    M_factor_permutation = random.choice(factor_permutations)
    N_factor_permutation = random.choice(factor_permutations)
    K_factor_permutation = random.choice(factor_permutations)
    
    dataflow_permutation = random.choice(dataflow_permutations)
    
    config = dict(
        matrix_size_M_dim=matrix_size,
        matrix_size_K_dim=matrix_size,
        matrix_size_N_dim=matrix_size,
        
        DRAM_factor_M=M_factor_permutation["DRAM"],
        DRAM_factor_N=N_factor_permutation["DRAM"],
        DRAM_factor_K=K_factor_permutation["DRAM"],
        DRAM_permutation=dataflow_permutation["DRAM"],
    
        ClusterArray_factor_M=M_factor_permutation["ClusterArray"],
        ClusterArray_factor_N=N_factor_permutation["ClusterArray"],
        ClusterArray_factor_K=K_factor_permutation["ClusterArray"],
        ClusterArray_permutation=dataflow_permutation["ClusterArray"],
    
        # GLB_Cluster_factor_M=M_factor_permutation["GLB_Cluster"],
        # GLB_Cluster_factor_N=N_factor_permutation["GLB_Cluster"],
        # GLB_Cluster_factor_K=K_factor_permutation["GLB_Cluster"],
        # GLB_Cluster_permutation=dataflow_permutation["GLB_Cluster"],
    
        # glb_factor_M=M_factor_permutation["glb"],
        # glb_factor_N=N_factor_permutation["glb"],
        # glb_factor_K=K_factor_permutation["glb"],
        # glb_permutation=dataflow_permutation["glb"],
    
        PE_Cluster_factor_M=M_factor_permutation["PE_Cluster"],
        PE_Cluster_factor_N=N_factor_permutation["PE_Cluster"],
        PE_Cluster_factor_K=K_factor_permutation["PE_Cluster"],
        PE_Cluster_permutation=dataflow_permutation["PE_Cluster"],
    
        iact_spad_factor_M=M_factor_permutation["iact_spad"],
        iact_spad_factor_N=N_factor_permutation["iact_spad"],
        iact_spad_factor_K=K_factor_permutation["iact_spad"],
        iact_spad_permutation=dataflow_permutation["iact_spad"],
    
        weight_spad_factor_M=M_factor_permutation["weight_spad"],
        weight_spad_factor_N=N_factor_permutation["weight_spad"],
        weight_spad_factor_K=K_factor_permutation["weight_spad"],
        weight_spad_permutation=dataflow_permutation["weight_spad"],
    
        psum_factor_M=M_factor_permutation["psum"],
        psum_factor_N=N_factor_permutation["psum"],
        psum_factor_K=K_factor_permutation["psum"],
        psum_permutation=dataflow_permutation["psum"],
    
        reg_factor_M=M_factor_permutation["reg"],
        reg_factor_N=N_factor_permutation["reg"],
        reg_factor_K=K_factor_permutation["reg"],
        reg_permutation=dataflow_permutation["reg"],
    
        density_weights=1,
        density_inputs=1
    )
    return config

In [103]:
num_of_random_samples = 2
visited = set()

# uses random sampling instead of a for loop
for sample in range(num_of_random_samples):
    # sample
    config = sample_config(dataflow_permutations, factor_permutations)
    while hash_config(config) in visited:
        config = sample_config(dataflow_permutations, factor_permutations)
    evaluate_config(config)

    visited.add(hash_config(config))

{'matrix_size_M_dim': 4096, 'matrix_size_K_dim': 4096, 'matrix_size_N_dim': 4096, 'DRAM_factor_M': 1, 'DRAM_factor_N': 64, 'DRAM_factor_K': 1, 'DRAM_permutation': ['N', 'K', 'M'], 'ClusterArray_factor_M': 1, 'ClusterArray_factor_N': 1, 'ClusterArray_factor_K': 1, 'ClusterArray_permutation': ['M', 'K', 'N'], 'PE_Cluster_factor_M': 1, 'PE_Cluster_factor_N': 1, 'PE_Cluster_factor_K': 1, 'PE_Cluster_permutation': ['M', 'K', 'N'], 'iact_spad_factor_M': 1, 'iact_spad_factor_N': 16, 'iact_spad_factor_K': 1, 'iact_spad_permutation': ['N', 'K', 'M'], 'weight_spad_factor_M': 128, 'weight_spad_factor_N': 1, 'weight_spad_factor_K': 128, 'weight_spad_permutation': ['N', 'K', 'M'], 'psum_factor_M': 32, 'psum_factor_N': 4, 'psum_factor_K': 32, 'psum_permutation': ['N', 'M', 'K'], 'reg_factor_M': 1, 'reg_factor_N': 1, 'reg_factor_K': 1, 'reg_permutation': ['K', 'M', 'N'], 'density_weights': 1, 'density_inputs': 1}


ValueError: Error loading YAML file /home/workspace/final_project/top.yaml.jinja2. Error loading YAML file /home/workspace/final_project/designs/baseline/baseline_mapping.yaml. 'GLB_Cluster_factor_M' is undefined

In [ ]:
!timeloop model ./output_dir/parsed-processed-input.yaml